# 1 马尔可夫奖励过程（Markov reward process,MRP）
## 在马尔可夫过程的基础上加入奖励函数$r$和折扣因子$\lambda$即可构成MRP。一个MRP由$(S,P,r,\lambda)$构成

## 在一个MRP中，从第t时刻状态开始，直到S_t终止状态时，所有奖励的衰减之和称为回报（Return）

## 该节计算了在给定状态转移概率矩阵P的情况下，从某个起始状态开始，到终止状态时的回报。

In [3]:
import numpy as np
np.random.seed(0)
# 定义状态转移概率矩阵P,一共有六个状态
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0]  # 定义奖励函数
gamma = 0.5  # 定义折扣因子


# 给定一条序列,计算从某个索引（起始状态）开始到序列最后（终止状态）得到的回报
def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        # 该for循环将range(0, 4)反向输出：3,2,1,0
        G = gamma * G + rewards[chain[i] - 1]  # 此处利用反向排序的方法解决了未来奖励随gamma的衰减。
    return G


# 一个状态序列,s1-s2-s3-s6
chain = [1, 2, 3, 6]   # 从S1出发，采样到的一个状态序列episode，采样生成该序列的概率和状态转移矩阵有关
start_index = 0
G = compute_return(start_index, chain, gamma)
print("根据本序列计算得到回报为：%s。" % G)

根据本序列计算得到回报为：-2.5。


# 2 价值函数的解析解求解方法，并据此计算上述马尔可夫奖励过程中，给定的状态转移概率矩阵P中所有状态的价值。

In [4]:
def compute(P, rewards, gamma, states_num):
    ''' 利用贝尔曼方程的矩阵形式计算解析解,states_num是MRP的状态数 '''
    rewards = np.array(rewards).reshape((-1, 1))  #将rewards写成列向量形式
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P),  # V=(I-\gamma*P)^(-1)*R
                   rewards)
    return value


V = compute(P, rewards, gamma, 6)
print("MRP中每个状态价值分别为\n", V)

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


# 3 给定一个MDP过程，并定义两个策略。
# 第一个策略是完全随机策略，即在每个状态下，智能体会以同样概率选取它可能采取的动作。例如，在s1下，智能体会以0.5和0.5的概率选取动作“保持s1”和“前往s2”。
# 第二个策略是一个提前设定的一个策略。

In [5]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}


# 把输入的两个字符串通过“-”连接,便于使用上述定义的P、R变量
def join(str1, str2):
    return str1 + '-' + str2


# 3.2 计算用随机策略（也就是代码中的Pi_1）时的状态价值函数

In [6]:
gamma = 0.5
# 转化后的MRP的状态转移矩阵
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


In [7]:
Pi_1.get(join(1, a_opt),0)
print(S)

['s1', 's2', 's3', 's4', 's5']


In [16]:
s=S[1]
for a in A:
    a=Pi_1.get(join(s, a),0)
    print(a)

0
0.5
0
0.5
0
0
0


In [15]:
join(s, a)

's2-概率前往'

# 4 蒙特卡洛方法（Monte-Carlo methods）

In [17]:
def sample(MDP, Pi, timestep_max, number):
    ''' 采样函数,策略Pi,限制最长时间步timestep_max,总共采样序列数number '''
    S, A, P, R, gamma = MDP
    episodes = []
    # 进行for循环采样，直到采样到目标数量的序列
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)]  # 随机选择一个除s5以外的状态s作为起点,str
        # 当前状态为终止状态或者时间步太长时,对一个序列的采样结束，然后回到外层for循环进行下一个序列的采样
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            # temp是概率
            rand, temp = np.random.rand(), 0   # 在[0, 1)中随机采样一个数,int
            # 在状态s下根据某个策略确定选择一个动作（对动作for高于该策略下选择该动作的的概率即为确定），
            # 然后保存记录当前选择的动作和当前状态下选择该动作的奖励之后结束循环
            for a_opt in A:
                # get取出的是选择某个策略的概率。
                # 这里之所以用+=是因为每个状态下只有两个动作，每个概率为0.5，不选这个就是选那个
                temp += Pi.get(join(s, a_opt), 0)  # 
                if temp > rand:
                    # 保存记录当前选择的动作和当前状态下选择该动作的奖励
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            rand, temp = np.random.rand(), 0
            # 根据状态转移概率得到下一个状态s_next
            for s_opt in S:
                # 从状态转移矩阵中获得在当前状态下选择当前动作后，进入下个某个状态的概率
                temp += P.get(join(join(s, a), s_opt), 0) 
                if temp > rand:
                    # 更新当前状态
                    s_next = s_opt
                    break
            # 将本次while循环采样得到的一个状态转移元组（s,a,r,s_next）放入当前序列中
            episode.append((s, a, r, s_next))  
            # 更新当前状态,开始接下来的while循环继续为本序列进行采样，直到满足该序列的采样结束条件
            s = s_next  
        episodes.append(episode)
    return episodes



# 在所有采样序列中计算每一个状态的价值
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
         #一个序列从后往前计算，即倒序，如range(2,-1,-1)返回三个数——2,1,0
        for i in range(len(episode) - 1, -1, -1): 
            (s, a, r, s_next) = episode[i]
#             print(s, a, r, s_next,'i',i,len(episode) - 1,'episode',episode,'*'*100)
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]


            

# 采样5次,每个序列最长不超过20步
episodes = sample(MDP, Pi_1, 20, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])
# 每条序列的最大采样次数
timestep_max = 20 
# 采样1000条序列,可以自行修改
num_epsiodes=1000 
episodes = sample(MDP, Pi_1, timestep_max, num_epsiodes)
gamma = 0.5
# 每个状态的状态价值函数
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma)
print("使用蒙特卡洛方法计算MDP的状态价值为\n", V)

第一条序列
 [('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s5', 0, 's5')]
第二条序列
 [('s4', '概率前往', 1, 's4'), ('s4', '前往s5', 10, 's5')]
第五条序列
 [('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]
使用蒙特卡洛方法计算MDP的状态价值为
 {'s1': -1.228923788722258, 's2': -1.6955696284402704, 's3': 0.4823809701532294, 's4': 5.967514743019431, 's5': 0}


# 5 占用度量

In [31]:
def occupancy(episodes, s, a, timestep_max, gamma):
    ''' 计算状态动作对（s,a）出现的频率,以此来估算策略的占用度量 '''
    rho = 0
    # 记录每个时间步t各被经历过几次，即在采样序列时，对每个序列的最大采样次数，即每个序列中最多有多少个元组
    total_times = np.zeros(timestep_max)  
    occur_times = np.zeros(timestep_max)  # 记录(s_t,a_t)=(s,a)的次数
    # 循环遍历每个序列
    for episode in episodes:
        # 一个序列里面有多个元组
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            total_times[i] += 1
            # 当该序列中的元组出现目标状态动作对(s,a)时，记录
            if s == s_opt and a == a_opt:
                occur_times[i] += 1
    for i in reversed(range(timestep_max)):
        if total_times[i]:
            rho += gamma**i * occur_times[i] / total_times[i]
    return (1 - gamma) * rho


gamma = 0.5
timestep_max = 1000

episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

0.10395776401475135 0.24711299717563512
